## 유기동물 데이터를 이용한 입양률 예측

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split,RandomizedSearchCV, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn import metrics, preprocessing
from scipy.stats import itemfreq
import mglearn

C:\Users\Bit\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


### 1. Data Loading

In [2]:
df8=pd.read_csv("D:\KSA\project_imsi\lostAnimal_20180101_20181231_vol3.csv", encoding="euc-kr")
df7=pd.read_csv("D:\KSA\project_imsi\lostAnimal_20170101_20171231_vol3.csv", encoding="euc-kr")
df6=pd.read_csv("D:\KSA\project_imsi\lostAnimal_20160101_20161231_vol3.csv", encoding="euc-kr")
df5=pd.read_csv("D:\KSA\project_imsi\lostAnimal_20150101_20151231_vol3.csv", encoding="euc-kr")

In [3]:
df5 = df5.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1'])
df6 = df6.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1'])
df7 = df7.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1'])
df8 = df8.drop(columns=['Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1'])

In [4]:
df5.tail(1)

,age(before),age(after),careAddr,careNm,careTel,chargeNm,colorCd,desertionNo,filename,happenDt,...,sexCd_Q,neuterYn_Y,neuterYn_N,neuterYn_U,careNm_ETC,careNm_H,careNm_C,careNm_O,careNm_AD,careNm_CM
79744,2달추정(년생),0.0,경기도 수원시 팔달구 인계동 1135-7,정동물병원,031-239-7585,노선영,흑백,4.413740e+14,http://www.animal.go.kr/files/shelter/2014/12/...,2015-01-01,...,False,False,True,False,False,True,False,False,False,False


In [5]:
df6.tail(1)

,age(before),age(after),careAddr,careNm,careTel,chargeNm,colorCd,desertionNo,filename,happenDt,...,sexCd_Q,neuterYn_Y,neuterYn_N,neuterYn_U,careNm_ETC,careNm_H,careNm_C,careNm_O,careNm_AD,careNm_CM
88174,2개월추정(년생),0.0,경기도 안산시 상록구 청곡길 50 (부곡동) 한국야생동물보호협회,한국야생동물보호협회,031-296-0124,박태환,흑,4.414010e+14,http://www.animal.go.kr/files/shelter/2015/11/...,2016-01-01,...,False,False,True,False,False,False,False,False,False,True


In [6]:
df7.tail(1)

,age(before),age(after),careAddr,careNm,careTel,chargeNm,colorCd,desertionNo,filename,happenDt,...,sexCd_Q,neuterYn_Y,neuterYn_N,neuterYn_U,careNm_ETC,careNm_H,careNm_C,careNm_O,careNm_AD,careNm_CM
100722,2012(년생),5,서울특별시 양천구 목동 657번지 16호,강현림 동물병원,02-2642-9159,김영승,갈색,4.113140e+14,http://www.animal.go.kr/files/shelter/2016/10/...,2017-01-01,...,False,False,True,False,False,True,False,False,False,False


In [7]:
df8.tail(1)

,age(before),age(after),careAddr,careNm,careTel,chargeNm,colorCd,desertionNo,filename,happenDt,...,sexCd_Q,neuterYn_Y,neuterYn_N,neuterYn_U,careNm_ETC,careNm_H,careNm_C,careNm_O,careNm_AD,careNm_CM
80329,2015(년생),3,대구광역시 수성구 달구벌대로 2475 (범어동),제니스동물병원,053-756-8290,경제환경과,흰색,4.270000e+14,http://www.animal.go.kr/files/shelter/2017/12/...,2018-01-01,...,False,True,False,False,False,True,False,False,False,False


In [ ]:
print(len(df5.columns))
print(len(df6.columns))
print(len(df7.columns))
print(len(df8.columns))

### 2. Data PreProcessing

#### - 데이터프레임 합치기 (15,16,17년도)

In [ ]:
#df567 = pd.merge(df5,df6, how='outer')
df567 = pd.concat([df5,df6], ignore_index=True)
len(df567)

In [ ]:
df567 = pd.concat([df567,df7], ignore_index=True)
len(df567)

In [ ]:
df567.head(1)

In [ ]:
df567.tail(1)

In [ ]:
df567.columns

#### - 필요없는 컬럼 삭제

In [ ]:
df = df567.drop(columns=['age(before)','colorCd','careAddr', 'careNm', 'careTel', 'chargeNm',
                      'desertionNo', 'filename', 'happenDt', 'happenPlace', 'kindCd','noticeComment',
                       'noticeNo', 'noticeSdt','officetel', 'popfile', 'processState',
                       'weight(before)', 'specialMark','breed', 'breed_Pre','breed.1'
                      ])

df_t = df8.drop(columns=['age(before)','colorCd','careAddr', 'careNm', 'careTel', 'chargeNm',
                      'desertionNo', 'filename', 'happenDt', 'happenPlace', 'kindCd','noticeComment',
                       'noticeNo', 'noticeSdt','officetel', 'popfile', 'processState',
                       'weight(before)', 'specialMark','breed', 'breed_Pre'
                      ])

#'Unnamed: 0', 'Unnamed: 0.1', 'Unnamed: 0.1.1',

In [ ]:
df.tail(2)

In [ ]:
df_t.tail(2)

In [ ]:
print("15/16/17 데이터프레임 사용할 컬럼 갯수 : ",len(df.columns))
print("컬럼 이름 : ", df.columns)

print("18 데이터프레임 사용할 컬럼 갯수 : ",len(df_t.columns))
print("컬럼 이름 : ", df_t.columns)

In [ ]:
df.info()

In [ ]:
df_t.info()

#### - 결측치 처리

In [ ]:
## 15/16/17년도 데이터프레임

#print(df.isnull().sum()) # 결측치 수
#df.isnull().any()
null_columns=df.columns[df.isnull().any()]
df[null_columns].isnull().sum()
#print(df[df["breed_Pre"].isnull()][null_columns])        

#print(df['weight(after)'].isnull().sum())       
#print(df['size'].isnull().sum())                

In [ ]:
## 18년도 데이터프레임
null_columns=df_t.columns[df_t.isnull().any()]
df_t[null_columns].isnull().sum()


In [ ]:
# weight(after) : 나이별로 묶어서 중위값넣기
df["weight(after)"].fillna(df.groupby("age_u")["weight(after)"].transform("median"), inplace=True)
df_t["weight(after)"].fillna(df_t.groupby("age_u")["weight(after)"].transform("median"), inplace=True)

In [ ]:
df.loc[pd.isnull(df["weight(after)"])]

In [ ]:
df_t.loc[pd.isnull(df["weight(after)"])]

In [ ]:
# size : 몸무게가 비어서 전처리 되지 않은 행 재전처리
df.loc[df['weight(after)'] <= 3, 'size'] = '초소형'
df.loc[(df['weight(after)'] > 3 ) & (df['weight(after)'] <=9 ), 'size'] = '소형'
df.loc[(df['weight(after)'] > 9 ) & (df['weight(after)'] <=25 ), 'size'] = '중형'
df.loc[df['weight(after)'] > 25, 'size'] = '대형'

df_t.loc[df_t['weight(after)'] <= 3, 'size'] = '초소형'
df_t.loc[(df_t['weight(after)'] > 3 ) & (df_t['weight(after)'] <=9 ), 'size'] = '소형'
df_t.loc[(df_t['weight(after)'] > 9 ) & (df_t['weight(after)'] <=25 ), 'size'] = '중형'
df_t.loc[df_t['weight(after)'] > 25, 'size'] = '대형'

In [ ]:
df.loc[pd.isnull(df["size"])]

In [ ]:
df_t.loc[pd.isnull(df_t["size"])]

In [ ]:
# 동물 종 : 몇개 안되니 기타로 입력
#df['breed_Pre'].fillna("기타",inplace=True)
#df.loc[pd.isnull(df["breed_Pre"])]

In [ ]:
null_columns=df.columns[df.isnull().any()]
df[null_columns].isnull().sum()

In [ ]:
null_columns=df_t.columns[df_t.isnull().any()]
df_t[null_columns].isnull().sum()

#### - neuterYn : 문자 -> 숫자

In [ ]:
neuter_mapping = {"Y":0,"N":1,"U":2}
df['neuterYn'] = df['neuterYn'].map(neuter_mapping)
df_t['neuterYn'] = df_t['neuterYn'].map(neuter_mapping)

print(df.neuterYn[1:3])
print(df_t.neuterYn[1:3])

#### - sexCd : 문자 -> 숫자

In [ ]:
sex_mapping = {"M":0,"F":1,"Q":2}
df['sexCd'] = df['sexCd'].map(sex_mapping)
df_t['sexCd'] = df_t['sexCd'].map(sex_mapping)

print(df.sexCd[1:3])
print(df_t.sexCd[1:3])

#### - orgNm(담당지역주소) : 두분류로 나눈뒤, 숫자 mapping

In [ ]:
df['sido'] = df['orgNm'].str.split(" ").str[0]
df_t['sido'] = df_t['orgNm'].str.split(" ").str[0]

print(df['sido'].value_counts())
print(df_t['sido'].value_counts())

In [ ]:
sido_mapping = {"경기도":0,"서울특별시":1,"부산광역시":2,"경상남도":3,
                "인천광역시":4,"충청남도":5,"강원도":6,"대구광역시":7,
                "전라북도":8,"경상북도":9,"대전광역시":10,"울산광역시":11,
                "충청북도":12,"전라남도":13,"제주특별자치도":14,"광주광역시":15,
                "세종특별자치시":16
}
df['sido'] = df['sido'].map(sido_mapping)
df_t['sido'] = df_t['sido'].map(sido_mapping)

print(df['sido'].head(3))
print(df_t['sido'].head(3))

#### - happenWd(발견요일) : 문자 -> 숫자 mapping 

In [ ]:
week_mapping = {"Monday":0, "Tuesday":2, "Wednesday":3, 
                "Thursday":4, "Friday":5, "Saturday":6, "Sunday":7}
df['happenWd'] = df['happenWd'].map(week_mapping)
df_t['happenWd'] = df_t['happenWd'].map(week_mapping) 

print(df['happenWd'].head(3))
print(df_t['happenWd'].head(3))

#### - size : 문자 -> 숫자 mapping

In [ ]:
#df['size'].fillna('중형',inplace=True)
size_mapping = {"대형":0,"소형":1,"중형":2,"초소형":3}
df['size'] = df['size'].map(size_mapping)
df_t['size'] = df_t['size'].map(size_mapping)

print(df['size'].head(3))
print(df_t['size'].head(3))

#### - age : 문자 -> 숫자 mapping 

In [ ]:
age_mapping = {"노견기":0,"성견기":1,"유견기":2}
df['age_u'] = df['age_u'].map(age_mapping)
df_t['age_u'] = df_t['age_u'].map(age_mapping)

print(df['size'].head(3))
print(df_t['size'].head(3))

#### - processState_Pre : 문자 -> 숫자 mapping 

In [ ]:
proc_mapping = {"C":0, "A":1, "D":2, "R":3, "E":4}
df['processState_Pre'] = df['processState_Pre'].map(proc_mapping)
df_t['processState_Pre'] = df_t['processState_Pre'].map(proc_mapping)

print(df['processState_Pre'].head(3))
print(df_t['processState_Pre'].head(3))

#### - DataFrame 분류 : 개, 고양이, 기타

In [ ]:
#개:0, 고양이:1, 기타:2
dfd = df[df.kind==0] #15/16/17
dfd2 = df_t[df_t.kind==0] #18
#dfc = df[df.kind==1]
#dfe = df[df.kind==2]

print(dfd['kind'].value_counts())
print(dfd2['kind'].value_counts())


### 3. Data Statistical Analysis

In [ ]:
dfd.describe()

In [ ]:
dfd2.describe()

In [ ]:
print(dfd.columns)
print(dfd2.columns)

In [ ]:
# processState와 상관관계
cor_t = np.round(dfd.corr(),3)
cor1 = cor_t.loc[:,[ 'processState_Pre', 'processState_C', 'processState_A',
       'processState_D', 'processState_R', 'processState_E']]
cor1

In [ ]:
cor_t2 = np.round(dfd2.corr(),3)
cor2 = cor_t2.loc[:,[ 'processState_Pre', 'processState_C', 'processState_A',
       'processState_D', 'processState_R', 'processState_E']]
cor2

### 4. Data Learning

In [ ]:
# null값 처리
# df.isnull().any()
#null_columns=df.columns[df.isnull().any()]
#df[null_columns].isnull().sum()
#print(df[df["breed_Pre"].isnull()][null_columns])

#### - feature 선택

In [ ]:
feature_name = ['age(after)','neuterYn','sexCd','weight(after)', 'kind', 'happenWd', 'happenMth', 
                'size', 'processState_Pre', 'sido']
df_Pre = dfd[feature_name]
dft_Pre = dfd2[feature_name]

feature_name2 = ['age(after)','neuterYn','sexCd','weight(after)', 'kind', 'happenWd', 'happenMth', 
                'size', 'processState_A', 'sido']
df_A = dfd[feature_name2]
dft_A = dfd2[feature_name2]

In [ ]:
df_Pre.tail(3)

In [ ]:
dft_Pre.tail(3)

In [ ]:
df_A.tail(3)

In [ ]:
dft_A.tail(3)

In [ ]:
# feature_name = ['kind', 'happenWd', 'happenMth','size','age_u', 'sexCd_M',
#                'sexCd_F', 'sexCd_Q', 'neuterYn_Y', 'neuterYn_N', 'neuterYn_U',
#                'careNm_ETC', 'careNm_H', 'careNm_C', 'careNm_O', 'careNm_AD',
#                'careNm_CM', 'sido','processState_Pre', 'sido']

# df_Pre = dfd[feature_name]
# dft_Pre = dfd2[feature_name]

# feature_name2 = ['kind', 'happenWd', 'happenMth','size','age_u', 'sexCd_M',
#                'sexCd_F', 'sexCd_Q', 'neuterYn_Y', 'neuterYn_N', 'neuterYn_U',
#                'careNm_ETC', 'careNm_H', 'careNm_C', 'careNm_O', 'careNm_AD',
#                'careNm_CM', 'sido','processState_A', 'sido']
# df_A = dfd[feature_name2]
# dft_A = dfd2[feature_name2]


In [ ]:
#df_Pre.head(3)

In [ ]:
#dft_Pre.head(3)

In [ ]:
#df_A.head(3)

In [ ]:
#dft_A.head(3)

#### - 종속변수 독립변수 추출

###### 1) processState_Pre 기준

In [ ]:
X=np.array(df_Pre.drop(columns='processState_Pre')) #종속변수
Y=np.array(df_Pre.processState_Pre) #독립변수

X2=np.array(dft_Pre.drop(columns='processState_Pre')) #종속변수
Y2=np.array(dft_Pre.processState_Pre) #독립변수

In [ ]:
print(df['processState_Pre'].value_counts())
table = itemfreq(Y)
x_ticks = ['0','1','2','3','4'] 
# 0 : 보호중, 1:입양,기증, 2:자연사,안락사, 3:반환, 4:방사,미포획
plt.bar(x_ticks, table[:,1], color='skyblue')
table[:,1]

###### 2) processState_A 기준

In [ ]:
Z=np.array(df_A.drop(columns='processState_A')) #종속변수
Q=np.array(df_A.processState_A) #독립변수
Z2=np.array(dft_A.drop(columns='processState_A')) #종속변수
Q2=np.array(dft_A.processState_A) #독립변수

In [ ]:
print(df_A['processState_A'].value_counts())
table = itemfreq(Q)
x_ticks = ['0','1'] # 0 : 입양X, 1:입양O
plt.bar(x_ticks, table[:,1], color='pink')
table[:,1]

##### - 데이터셋 나누기

In [ ]:
# 15/16/17 train 데이터 
print(X.shape)
print(Y.shape)
print(Z.shape)
print(Q.shape)

In [ ]:
#18년도 test 데이터
print(X2.shape)
print(Y2.shape)
print(Z2.shape)
print(Q2.shape)

In [ ]:
#X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.3, random_state=5)
#Z_train, Z_test, Q_train, Q_test = train_test_split(Z,Q,test_size=0.3, random_state=5)

# train_test_split
# : 데이터를  train set과 test set으로 단순분리
# : 원래 트레이닝 데이터와 결과값이 X와 Y를 무작위로 섞은 후 test_size로 주어진 비율만큼 나눔
# test_size 0.3은 X와 Y를 무작위로 섞은 후에 70%은 train으로, 30%는 test로 하는것
# random_state : 난수발생을 위한 seed의 인자값

#### - 모형 적용

##### KNN

In [ ]:
# knn객체 : 훈련데이터로 모델을 만들고 새로운 데이터포인트에 대해 예측하는 알고리즘을 캡슐화
# 또한 알고리즘이 훈련데이터로부터 추출한 정보를 담고 있음
# KNeighborsClassifier : 훈련데이터 자체를 저장
knn_Pre = KNeighborsClassifier(n_neighbors=1) # n_neighbor : 이웃의 갯수
knn_A = KNeighborsClassifier(n_neighbors=1) # n_neighbor : 이웃의 갯수
print(knn_Pre)
print(knn_A)
# fit() : 훈련데이터셋으로부터 모델 만들기, knn객체 자체를 반환, 그리고 knn객체 자체를 변경시킴, knn객체가 문자열 형태로 출력됨
#출력에서 모델으르생성할떄 사용한 매개변수 확인 가능
# 훈련데이터(Numpy배열 X_train), 훈련데이터레이블(Numpy배열 y_train)을 매개변수로 받음
knn_Pre.fit(X, Y)
knn_A.fit(Z, Q)

##### Logistic Regression

In [ ]:
glm_Pre = LogisticRegression()
glm_A = LogisticRegression()

glm_Pre.fit(X, Y)
glm_A.fit(Z, Q)

##### Naive Bayes

In [ ]:
GNB_Pre = GaussianNB()
GNB_A = GaussianNB()
print(GNB_Pre)
print(GNB_A)

GNB_Pre.fit(X, Y)
GNB_A.fit(Z, Q) 

##### Tree

In [ ]:
depth_grid = np.arange(2,31,2)
parameters = {'max_depth':depth_grid}

gridCV_Pre = GridSearchCV(DecisionTreeClassifier(), parameters, cv=10)
gridCV_A = GridSearchCV(DecisionTreeClassifier(), parameters, cv=10)

gridCV_Pre.fit(X, Y);
gridCV_A.fit(Z, Q);

best_depth_Pre = gridCV_Pre.best_params_['max_depth']
best_depth_A = gridCV_A.best_params_['max_depth']

print("Tree best depth : " + str(best_depth_Pre))
print("Tree best depth : " + str(best_depth_A))

In [ ]:
DTC_best_Pre = DecisionTreeClassifier(max_depth=best_depth_Pre)
DTC_best_A = DecisionTreeClassifier(max_depth=best_depth_A)

DTC_best_Pre.fit(X, Y);
DTC_best_A.fit(Z, Q);

##### Random Forest

In [ ]:
estimator_grid = np.arange(1, 30, 5)
depth_grid = np.arange(1, 10, 2)
parameters = {'n_estimators': estimator_grid, 'max_depth': depth_grid}

gridCV_Pre = GridSearchCV(RandomForestClassifier(), param_grid=parameters, cv=10)
gridCV_A = GridSearchCV(RandomForestClassifier(), param_grid=parameters, cv=10)

gridCV_Pre.fit(X, Y);
gridCV_A.fit(Z, Q);

best_n_estim_Pre = gridCV_Pre.best_params_['n_estimators']
best_depth_Pre = gridCV_Pre.best_params_['max_depth']

best_n_estim_A = gridCV_A.best_params_['n_estimators']
best_depth_A = gridCV_A.best_params_['max_depth']

In [ ]:
print("Random Forest best n estimator : " + str(best_n_estim_Pre))
print("Random Forest best depth : " + str(best_depth_Pre))

print("Random Forest best n estimator : " + str(best_n_estim_A))
print("Random Forest best depth : " + str(best_depth_A))

In [ ]:
RF_best_Pre = RandomForestClassifier(max_depth=best_depth_Pre,n_estimators=best_n_estim_Pre,random_state=3)
RF_best_A = RandomForestClassifier(max_depth=best_depth_A,n_estimators=best_n_estim_A,random_state=3)

RF_best_Pre.fit(X, Y);
RF_best_A.fit(Z, Q);

##### AdaBoost

In [ ]:
estimator_grid = np.arange(30, 80, 10)
learning_rate_grid = np.array([0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9])
parameters = {'n_estimators': estimator_grid, 'learning_rate': learning_rate_grid}

gridCV_Pre = GridSearchCV(AdaBoostClassifier(), param_grid=parameters, cv=10)
gridCV_A = GridSearchCV(AdaBoostClassifier(), param_grid=parameters, cv=10)

gridCV_Pre.fit(X, Y)
gridCV_A.fit(Z, Q)

best_n_estim_Pre = gridCV_Pre.best_params_['n_estimators']
best_learn_rate_Pre = gridCV_Pre.best_params_['learning_rate']

best_n_estim_A = gridCV_A.best_params_['n_estimators']
best_learn_rate_A = gridCV_A.best_params_['learning_rate']

In [ ]:
print("Ada Boost best n estimator : " + str(best_n_estim_Pre))
print("Ada Boost best learning rate : " + str(best_learn_rate_A))

print("Ada Boost best n estimator : " + str(best_n_estim_Pre))
print("Ada Boost best learning rate : " + str(best_learn_rate_A))

In [ ]:
AB_best_Pre = AdaBoostClassifier(n_estimators=best_n_estim_Pre,learning_rate=best_learn_rate_Pre,random_state=3)
AB_best_A = AdaBoostClassifier(n_estimators=best_n_estim_A,learning_rate=best_learn_rate_A,random_state=3)

AB_best_Pre.fit(X, Y);
AB_best_A.fit(Z, Q);

##### SVM (Support Vector Machine)

In [ ]:
C_grid = [0.001, 0.01, 0.1, 1, 10]
gamma_grid = [0.001, 0.01, 0.1, 1]
parameters = {'C': C_grid, 'gamma' : gamma_grid}

gridCV_Pre = GridSearchCV(SVC(kernel='rbf'), parameters, cv=10);
gridCV_Pre.fit(X, Y)

gridCV_A = GridSearchCV(SVC(kernel='rbf'), parameters, cv=10);
gridCV_A.fit(Z, Q)


best_C_Pre = gridCV_Pre.best_params_['C']
best_gamma_Pre = gridCV_Pre.best_params_['gamma']

best_C_A = gridCV_A.best_params_['C']
best_gamma_A = gridCV_A.best_params_['gamma']

In [ ]:
print("SVM best C : " + str(best_C_Pre))
print("SVM best gamma : " + str(best_gamma_Pre))

print("SVM best C : " + str(best_C_A))
print("SVM best gamma : " + str(best_gamma_A))

In [ ]:
SVM_best_Pre = SVC(C=best_C_Pre,gamma=best_gamma_Pre)
SVM_best_Pre.fit(X, Y);

SVM_best_A = SVC(C=best_C_A,gamma=best_gamma_A)
SVM_best_A.fit(Z, Q);

#### - 예측 및 검증

In [ ]:
X_new = np.array([[5,1,0,4.5,0,1,11,2,12]])
Z_new = np.array([[5,1,0,4.5,0,1,11,2,12]])
print("X_new.shape : {}".format(X_new.shape))
print("Z_new.shape : {}".format(Z_new.shape))

In [ ]:
# X_new = np.array([[0,3,11,2,1,0,0,1,1,0,0,0,0,0,1,0,0,4,3]])
# Z_new = np.array([[0,3,11,2,1,0,0,1,1,0,0,0,0,0,1,0,0,4,3]])
# print("X_new.shape : {}".format(X_new.shape))
# print("Z_new.shape : {}".format(Z_new.shape))

In [ ]:
target_Pre = {0:'보호중', 1:'입양기증', 2:'자연사안락사', 3:'반환', 4:'방사미포획'}
target_A = {0:'입양X', 1:'입양O'}

##### KNN

In [ ]:
prediction_Pre = knn_Pre.predict(X_new)
print("예측:{}".format(prediction_Pre))
p_Pre = int(prediction_Pre)
print("예측한 타깃의 이름 : {}".format(target_Pre[p_Pre]))

prediction_A = knn_A.predict(X_new)
print("예측:{}".format(prediction_A))
p_A = int(prediction_A)
print("예측한 타깃의 이름 : {}".format(target_A[p_A]))

In [ ]:
#테스트데이터 사용 (모델을 만들때 사용하지 않았고, 테스트셋에 있는 각 붓꽃의 품종을 정확히 알고있음)
Y_pred = knn_Pre.predict(X2) 
print("테스트 셋에 대한 예측값: {}".format(Y_pred))

Q_pred = knn_A.predict(Z2) 
print("테스트 셋에 대한 예측값: {}".format(Q_pred))

In [ ]:
print( "KNN best accuracy : " + str(np.round(metrics.accuracy_score(Y2,Y_pred),3)))
print( "KNN best accuracy : " + str(np.round(metrics.accuracy_score(Q2,Q_pred),3)))

##### Logistic Regression

In [ ]:
prediction_Pre = glm_Pre.predict(X_new)
print("예측:{}".format(prediction_Pre))
p_Pre = int(prediction_Pre)
print("예측한 타깃의 이름 : {}".format(target_Pre[p_Pre]))

prediction_A = glm_A.predict(X_new)
print("예측:{}".format(prediction_A))
p_A = int(prediction_A)
print("예측한 타깃의 이름 : {}".format(target_A[p_A]))

In [ ]:
#테스트데이터 사용 (모델을 만들때 사용하지 않았고, 테스트셋에 있는 각 붓꽃의 품종을 정확히 알고있음)
Y_pred = glm_Pre.predict(X2) 
print("테스트 셋에 대한 예측값: {}".format(Y_pred))

Q_pred = glm_A.predict(Z2) 
print("테스트 셋에 대한 예측값: {}".format(Q_pred))

In [ ]:
print( "Logistic Regression best accuracy : " + str(np.round(metrics.accuracy_score(Y2,Y_pred),3)))
print( "Logistic Regression best accuracy : " + str(np.round(metrics.accuracy_score(Q2,Q_pred),3)))

##### Naive Bayes

In [ ]:
prediction_Pre = GNB_Pre.predict(X_new)
print("예측:{}".format(prediction_Pre))
p_Pre = int(prediction_Pre)
print("예측한 타깃의 이름 : {}".format(target_Pre[p_Pre]))

prediction_A = GNB_A.predict(X_new)
print("예측:{}".format(prediction_A))
p_A = int(prediction_A)
print("예측한 타깃의 이름 : {}".format(target_A[p_A]))

In [ ]:
#테스트데이터 사용 (모델을 만들때 사용하지 않았고, 테스트셋에 있는 각 붓꽃의 품종을 정확히 알고있음)
Y_pred = GNB_Pre.predict(X2) 
print("테스트 셋에 대한 예측값: {}".format(Y_pred))

Q_pred = GNB_A.predict(Z2) 
print("테스트 셋에 대한 예측값: {}".format(Q_pred))

In [ ]:
print( "Naive Bayes best accuracy : " + str(np.round(metrics.accuracy_score(Y2,Y_pred),3)))
print( "Naive Bayes best accuracy : " + str(np.round(metrics.accuracy_score(Q2,Q_pred),3)))

##### Tree

In [ ]:
prediction_Pre = DTC_best_Pre.predict(X_new)
print("예측:{}".format(prediction_Pre))
p_Pre = int(prediction_Pre)
print("예측한 타깃의 이름 : {}".format(target_Pre[p_Pre]))

prediction_A = DTC_best_A.predict(X_new)
print("예측:{}".format(prediction_A))
p_A = int(prediction_A)
print("예측한 타깃의 이름 : {}".format(target_A[p_A]))

In [ ]:
#테스트데이터 사용 (모델을 만들때 사용하지 않았고, 테스트셋에 있는 각 붓꽃의 품종을 정확히 알고있음)
Y_pred = DTC_best_Pre.predict(X2) 
print("테스트 셋에 대한 예측값: {}".format(Y_pred))

Q_pred = DTC_best_A.predict(Z2) 
print("테스트 셋에 대한 예측값: {}".format(Q_pred))

In [ ]:
print( "Tree best accuracy : " + str(np.round(metrics.accuracy_score(Y2,Y_pred),3)))
print( "Tree best accuracy : " + str(np.round(metrics.accuracy_score(Q2,Q_pred),3)))

##### Random Forest

In [ ]:
prediction_Pre = RF_best_Pre.predict(X_new)
print("예측:{}".format(prediction_Pre))
p_Pre = int(prediction_Pre)
print("예측한 타깃의 이름 : {}".format(target_Pre[p_Pre]))

prediction_A = RF_best_A.predict(X_new)
print("예측:{}".format(prediction_A))
p_A = int(prediction_A)
print("예측한 타깃의 이름 : {}".format(target_A[p_A]))

In [ ]:
#테스트데이터 사용 (모델을 만들때 사용하지 않았고, 테스트셋에 있는 각 붓꽃의 품종을 정확히 알고있음)
Y_pred =RF_best_Pre.predict(X2) 
print("테스트 셋에 대한 예측값: {}".format(Y_pred))

Q_pred = RF_best_A.predict(Z2) 
print("테스트 셋에 대한 예측값: {}".format(Q_pred))

In [ ]:
print( "Random Forest best accuracy : " + str(np.round(metrics.accuracy_score(Y2,Y_pred),3)))
print( "Random Forest best accuracy : " + str(np.round(metrics.accuracy_score(Q2,Q_pred),3)))

##### AdaBoost

In [ ]:
prediction_Pre = AB_best_Pre.predict(X_new)
print("예측:{}".format(prediction_Pre))
p_Pre = int(prediction_Pre)
print("예측한 타깃의 이름 : {}".format(target_Pre[p_Pre]))

prediction_A = AB_best_A.predict(X_new)
print("예측:{}".format(prediction_A))
p_A = int(prediction_A)
print("예측한 타깃의 이름 : {}".format(target_A[p_A]))

In [ ]:
#테스트데이터 사용 (모델을 만들때 사용하지 않았고, 테스트셋에 있는 각 붓꽃의 품종을 정확히 알고있음)
Y_pred =AB_best_Pre.predict(X2) 
print("테스트 셋에 대한 예측값: {}".format(Y_pred))

Q_pred = AB_best_A.predict(Z2) 
print("테스트 셋에 대한 예측값: {}".format(Q_pred))

In [ ]:
print( "Ada Boost best accuracy : " + str(np.round(metrics.accuracy_score(Y2,Y_pred),3)))
print( "Ada Boost best accuracy : " + str(np.round(metrics.accuracy_score(Q2,Q_pred),3)))

##### SVM (Support Vector Machine)

In [ ]:
prediction_Pre = SVM_best_Pre.predict(X_new)
print("예측:{}".format(prediction_Pre))
p_Pre = int(prediction_Pre)
print("예측한 타깃의 이름 : {}".format(target_Pre[p_Pre]))

prediction_A = SVM_best_A.predict(X_new)
print("예측:{}".format(prediction_A))
p_A = int(prediction_A)
print("예측한 타깃의 이름 : {}".format(target_A[p_A]))

In [ ]:
#테스트데이터 사용 (모델을 만들때 사용하지 않았고, 테스트셋에 있는 각 붓꽃의 품종을 정확히 알고있음)
Y_pred =SVM_best_Pre.predict(X2) 
print("테스트 셋에 대한 예측값: {}".format(Y_pred))

Q_pred = SVM_best_A.predict(Z2) 
print("테스트 셋에 대한 예측값: {}".format(Q_pred))

In [ ]:
print( "SVM best accuracy : " + str(np.round(metrics.accuracy_score(Y2,Y_pred),3)))
print( "SVM best accuracy : " + str(np.round(metrics.accuracy_score(Q2,Q_pred),3)))

##### - 검증2

In [ ]:
# 예측값 y_pred와 기존의 정답인 y_test가 맞는지 확인
print("테스트 셋에 대한 정확도 : {:.2f}".format(np.mean(Y_pred == Y_test)))
print("테스트 셋에 대한 정확도 : {:.2f}".format(np.mean(Z_pred == Z_test)))

In [ ]:
print("테스트 셋트에 대한 정확도 : {:.2f}".format(knn_Pre.score(X_test, Y_test)))
print("테스트 셋트에 대한 정확도 : {:.2f}".format(knn_A.score(Z_test, Q_test)))